# Немного о развитии индустрии игр в начале XXI века

- Автор: Щербань Сергей
- Дата: 24.06.2025

### Цели и задачи проекта

<font color='#777778'>Категоризовать игры вышедшие с 2000 по 2013 год по оценкам пользователей и экспертов, и выделить топ-7 платформ по количеству игр.</font>

### Описание данных

<font color='#777778'>Данные /datasets/new_games.csv содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:
    
- Name — название игры.
- Platform — название платформы.
- Year of Release — год выпуска игры.
- Genre — жанр игры.
- NA sales — продажи в Северной Америке (в миллионах проданных копий).
- EU sales — продажи в Европе (в миллионах проданных копий).
- JP sales — продажи в Японии (в миллионах проданных копий).
- Other sales — продажи в других странах (в миллионах проданных копий).
- Critic Score — оценка критиков (от 0 до 100).
- User Score — оценка пользователей (от 0 до 10).
- Rating — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.
</font>

### Содержимое проекта

<font color='#777778'>Проект состоит из этапов:
    
- Загрузка и знакомство с данными
    
- Проверка ошибок в данных и их предобработка
    
- Фильтрация данных
    
- Категоризация данных
    
- Вывод
</font>

---

## 1. Загрузка данных и знакомство с ними

- Загружаем необходимые библиотеки Python и данные датасета `/datasets/new_games.csv`.


In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

- Знакомимся с данными: выводим первые строки и результат метода `info()`.


In [4]:
display(df.head(5))

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
,RangeIndex: 16956 entries, 0 to 16955
,Data columns (total 11 columns):
, #   Column           Non-Null Count  Dtype  
,---  ------           --------------  -----  
, 0   Name             16954 non-null  object 
, 1   Platform         16956 non-null  object 
, 2   Year of Release  16681 non-null  float64
, 3   Genre            16954 non-null  object 
, 4   NA sales         16956 non-null  float64
, 5   EU sales         16956 non-null  object 
, 6   JP sales         16956 non-null  object 
, 7   Other sales      16956 non-null  float64
, 8   Critic Score     8242 non-null   float64
, 9   User Score       10152 non-null  object 
, 10  Rating           10085 non-null  object 
,dtypes: float64(4), object(7)
,memory usage: 1.4+ MB


- В датафрейме 16956 строк и 11 столбцов.
- Пропуски встречаются в 6/11 столбцах.
- Year of Release - тип данных float64, вместо object, но для удобного анализа переведем в int64
- EU sales, JP sales - тип данных object, изменим тип на float64

Названия столбцов корректно отображают содержимое данных, но для далее будут приведены к стилю snake_case

---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

- Выводим на экран названия всех столбцов датафрейма и проверяем их стиль написания.
- Приводим все столбцы к стилю snake_case. Названия в нижнем регистре, а вместо пробелов — подчёркивания.

In [6]:
print("Исходные названия столбцов:")
display(df.columns.tolist())

Исходные названия столбцов:


['Name',
 'Platform',
 'Year of Release',
 'Genre',
 'NA sales',
 'EU sales',
 'JP sales',
 'Other sales',
 'Critic Score',
 'User Score',
 'Rating']

In [7]:
# Приводим к стилю snake_case
df.columns = (
    df.columns
    .str.lower()
    .str.replace(' ', '_')
)
print("\nНовые названия столбцов (стиль snake_case):")
display(df.columns.tolist())


,Новые названия столбцов (стиль snake_case):


['name',
 'platform',
 'year_of_release',
 'genre',
 'na_sales',
 'eu_sales',
 'jp_sales',
 'other_sales',
 'critic_score',
 'user_score',
 'rating']

### 2.2. Типы данных

- Встречаются некорректные типы данных, предположим их причины.
- При необходимости проводим преобразование типов данных. Сначала обрабатываем пропуски, а затем преобразовываем типы данных.

Иногда Pandas автоматически преобразует столбцы в тип данных при обнаружении пропущенных значений или аномалий в данных.
Поэтому сказать о точной причине смены с object/int64 на float64 нет возможности, могло быть и так, что данные представлены как числа с плавающей точкой изначально.

In [8]:
# Считаем количество строк до удаления данных
initial_rows_count = len(df)

# Удаляем строки с пропусками в "Название игры" и "Год выпуска игры" - их мало и такие значения не вычислить
df = df.dropna(subset=['name', 'year_of_release'])

# Меняем тип данных для "Год выпуска игры" на целочисленный

df['year_of_release'] = df['year_of_release'].astype('int64')

- В числовых столбцах встретились строковые значения, например `unknown` или другие. Приводим такие столбцы к числовому типу данных, заменив строковые значения на пропуски.

In [9]:
# Меняем строковые значения на пропуски для eu_sales, jp_sales и user_score
df['eu_sales'] = df['eu_sales'].replace(['unknown'], np.nan).astype(float)
df['jp_sales'] = df['jp_sales'].replace(['unknown'], np.nan).astype(float)
df['user_score'] = df['user_score'].replace(['tbd'], np.nan).astype(float)

### 2.3. Наличие пропусков в данных

- Счтаем количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [10]:
print("Абсолютные значения пропусков:")
display(df.isna().sum() / len(df)) 

Абсолютные значения пропусков:


name               0.000000
platform           0.000000
year_of_release    0.000000
genre              0.000000
na_sales           0.000000
eu_sales           0.000360
jp_sales           0.000240
other_sales        0.000000
critic_score       0.515259
user_score         0.546855
rating             0.406379
dtype: float64

In [11]:
print("Относительные значения пропусков (%):")
display(round((df.isna().sum() / len(df) * 100), 2))

Относительные значения пропусков (%):


name                0.00
platform            0.00
year_of_release     0.00
genre               0.00
na_sales            0.00
eu_sales            0.04
jp_sales            0.02
other_sales         0.00
critic_score       51.53
user_score         54.69
rating             40.64
dtype: float64

- Изучаем данные с пропущенными значениями. Даем промежуточный вывод: для каких столбцов характерны пропуски и сколько их. Предпологаем, почему пропуски могли возникнуть. Указываем, какие действия с этими данными можно сделать.


Данные с пропущенными значениями:
- critic_score - 51.53 %
- user_score - 54.69 %
- rating - 40.64 %

Наличие пропусков не всегда говорит об ошибке. Если игра непопулярная, или не переведена на многие языки, она может не участвовать в рейтинге критиков (critic_score) и пользователей (user_score). Оценка пользователей и критиков - очень индивидуальный параметр, но терять почти 6000 из 13000 для 2000-2013 годов игры не хочется, поэтому приведем к среднему значению.

Другой вопрос, что у игры не заявлен рейтинг организации ESRB (rating) - это точно не полная информация. Но для анализа не используется, поэтому пропускаем.


Данные с количеством проданных копий игры в Европе (eu_sales) и Японии (jp_sales), можно заменить на среднее значение в зависимости от названия платформы (platform) и года выхода игры (year_of_release).

In [12]:
stats = df[['critic_score', 'user_score']].describe()
print(stats)

       critic_score   user_score
,count   8085.000000  7558.000000
,mean      68.958689     7.124438
,std       13.923424     1.498803
,min       13.000000     0.000000
,25%       60.000000     6.400000
,50%       71.000000     7.500000
,75%       79.000000     8.200000
,max       98.000000     9.700000


- Обработаем пропущенные значения. Для каждого случая попробуем выбрать оптимальный, на наш взгляд, вариант: заменить на определённое значение, оставить как есть или удалить.
- Решаем заменить пропуски MNAR на значение-индикатор.

In [13]:
# Группируем данные по platform и year_of_release
grouped = df.groupby(['platform', 'year_of_release'])

# Вычисляем среднее значение для каждой группы
mean_values = grouped.mean()

# Заменяем пропуски средним значением для каждой группы
df['eu_sales'] = df['eu_sales'].fillna(df.groupby(['platform', 'year_of_release'])['eu_sales'].transform('mean'))
df['jp_sales'] = df['jp_sales'].fillna(df.groupby(['platform', 'year_of_release'])['jp_sales'].transform('mean'))

In [14]:
# Заменяем пропуски в столбцах оценки отриццательным значением, которое точно не встретится в реальных данных
df['critic_score'] = df['critic_score'].fillna(-1)
df['user_score'] = df['user_score'].fillna(-1)

# Меняем тип данных для "Оценка критиков" и "Оценка пользователей" на целочисленный
df['critic_score'] = df['critic_score'].astype('int64')
df['user_score'] = df['user_score'].astype('int64')

### 2.4. Явные и неявные дубликаты в данных

- Изучаем уникальные значения в категориальных данных, с названиями жанра игры, платформы, рейтинга и года выпуска. Проверяем, встречаются ли среди данных неявные дубликаты, связанные с опечатками или разным способом написания.
- При необходимости проведим нормализацию данных с текстовыми значениями. Названия или жанры игр приводим к нижнему регистру.

In [15]:
# Проверяем уникальные значения данных с типом object
for column in ['genre', 'platform', 'year_of_release']:
    unique_values = df[column].unique()
    sorted_values = sorted(unique_values)
    print(f"Уникальные значения в столбце '{column}':")
    print(sorted_values)
    print("\n")  # Добавляем пустую строку между выводами
print(f"Уникальные значения в столбце 'rating':")
unique_ratings = df['rating'].unique()
print(unique_ratings)

Уникальные значения в столбце 'genre':
,['ACTION', 'ADVENTURE', 'Action', 'Adventure', 'FIGHTING', 'Fighting', 'MISC', 'Misc', 'PLATFORM', 'PUZZLE', 'Platform', 'Puzzle', 'RACING', 'ROLE-PLAYING', 'Racing', 'Role-Playing', 'SHOOTER', 'SIMULATION', 'SPORTS', 'STRATEGY', 'Shooter', 'Simulation', 'Sports', 'Strategy']
,
,
,Уникальные значения в столбце 'platform':
,['2600', '3DO', '3DS', 'DC', 'DS', 'GB', 'GBA', 'GC', 'GEN', 'GG', 'N64', 'NES', 'NG', 'PC', 'PCFX', 'PS', 'PS2', 'PS3', 'PS4', 'PSP', 'PSV', 'SAT', 'SCD', 'SNES', 'TG16', 'WS', 'Wii', 'WiiU', 'X360', 'XB', 'XOne']
,
,
,Уникальные значения в столбце 'year_of_release':
,[1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
,
,
,Уникальные значения в столбце 'rating':
,['E' nan 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']


In [16]:
# Проводим названия и жанры игр к нижнему регистру
df['name'] = df['name'].str.lower()
df['genre'] = df['genre'].str.lower()

In [17]:
# Заменим значение рейтинга 'K-A' (Kids to Adults) на 'E' (Everyone)
df['rating'] = df['rating'].replace('K-A', 'E')

- Нормализовали данные и устраняем дубликаты.

In [18]:
# Сортируем датафрейм
df = df.sort_values(by=[ 'year_of_release', 'name', 'platform', 'genre'])

# Убираем не информативный Рейтинг
df = df.drop(columns=['rating'])

# Считаем дубликаты
count_duplicates = df.duplicated().sum()

# Убираем дубликаты
df = df.drop_duplicates()

- Пишем промежуточный вывод: указываем количество найденных дубликатов и действия по их обработке.

In [19]:
print(f"Количество дубликатов: {count_duplicates}")

Количество дубликатов: 235


Дубликаты были удалены по всему датафрейму. Оставлены первые встреченные уникальные строки.

- В процессе подготовки данных мы могли удалять, например строки с пропусками или ошибками, дубликаты и прочее. В этом случае посчитаем количество удалённых строк в абсолютном и относительном значениях.

In [20]:
# Считаем количество строк после обработки данных
final_rows_count = len(df)
deleted_rows = initial_rows_count - final_rows_count
relative_deleted_rows = (deleted_rows / initial_rows_count) * 100

print(f"Количество удалённых строк (абсолютное): {deleted_rows}")
print(f"Процент удалённых строк (относительное): {relative_deleted_rows:.2f}%")

Количество удалённых строк (абсолютное): 512
,Процент удалённых строк (относительное): 3.02%


- После проведения предобработки данных напишите общий промежуточный вывод.

In [21]:
unique_names_count = df['name'].nunique()
print(f"Теперь данные подготолвены для {unique_names_count} игр, на разных платформах с разным годом выпуска.")

Теперь данные подготолвены для 11426 игр, на разных платформах с разным годом выпуска.


---

## 3. Фильтрация данных

Для изучения истории продаж игр в начале XXI века, выберем интересующий период с 2000 по 2013 год включительно. Отберем данные по этому показателю. Сохраним новый срез данных в отдельном датафрейме - `df_actual`.

In [22]:
# Создаем новый датафрейм
# df_actual = df[(df['year_of_release'] >= 2000) & (df['year_of_release'] <= 2013)]
df_actual = df[df['year_of_release'].between(2000, 2013)]

---

## 4. Категоризация данных
    
Проведите категоризацию данных:
- Разделите все игры по оценкам пользователей и выделите такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [23]:
# Разбиваем на категории по оценкам пользователей
df_actual.loc[:, 'user_score_category'] = pd.cut(
    df_actual['user_score'],
    bins=[0, 3, 8, 10],
    labels=['низкая оценка', 'средняя оценка', 'высокая оценка'],
    right=False
)

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
,A value is trying to be set on a copy of a slice from a DataFrame.
,Try using .loc[row_indexer,col_indexer] = value instead
,
,See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
,  self.obj[key] = value
,/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
,A value is trying to be set on a copy of a slice from a DataFrame.
,Try using .loc[row_indexer,col_indexer] = value instead
,
,See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
,  self._setitem_single_column(ilocs[0], value, pi)


- Разделите все игры по оценкам критиков и выделите такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [24]:
# Разбиваем на категории по оценкам пользователей
df_actual.loc[:, 'critic_score_category'] = pd.cut(
    df_actual['critic_score'],
    bins=[0, 30, 80, 100],
    labels=['низкая оценка', 'средняя оценка', 'высокая оценка'],
    right=False
)

- После категоризации данных проверьте результат: сгруппируйте данные по выделенным категориям и посчитайте количество игр в каждой категории.

In [25]:
# Подсчитываем количество игр в каждой категории для оценок пользователей
user_scores_count = df_actual['user_score_category'].value_counts().reset_index()
user_scores_count.columns = ['user_score_category', 'count']

# Подсчитываем общее количество игр
total_games = df_actual.shape[0]

# Вычисляем проценты для оценок пользователей
user_scores_count['percentage'] = (user_scores_count['count'] / total_games) * 100

# Подсчитываем количество игр в каждой категории для оценок критиков
critic_scores_count = df_actual['critic_score_category'].value_counts().reset_index()
critic_scores_count.columns = ['critic_score_category', 'count']

# Вычисляем проценты для оценок критиков
critic_scores_count['percentage'] = (critic_scores_count['count'] / total_games) * 100

display(user_scores_count)
display(critic_scores_count)


,user_score_category,count,percentage
0,средняя оценка,4081,31.930209
1,высокая оценка,2286,17.885924
2,низкая оценка,116,0.907597


,critic_score_category,count,percentage
0,средняя оценка,5422,42.422346
1,высокая оценка,1692,13.238401
2,низкая оценка,55,0.430326


- Выделим топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [26]:
# Группируем данные по платформе и считаем количество игр для каждой платформы
platforms_count = df_actual.groupby('platform').size().reset_index(name='count')

# Сортируем платформы по количеству игр в descending порядке
platforms_count_sorted = platforms_count.sort_values(by='count', ascending=False)

# Выбираем топ-7 платформ
top_7_platforms = platforms_count_sorted.head(7)
top_7_platforms = top_7_platforms.reset_index(drop=True)
display(top_7_platforms)

,platform,count
0,PS2,2127
1,DS,2120
2,Wii,1275
3,PSP,1180
4,X360,1121
5,PS3,1087
6,GBA,811


Топ-7 платформ: Play Station 2, Nintendo DS, Wii, PlayStation Portable, Xbox 360, Play Station 3, Game Boy Advance

---

## 5. Итоговый вывод



С 2000 по 2013 год вышло 8820 игр на разных платформах. Количество игр выпущенных за год увеличелось на 258.57%, если сравнить 2000 и 2010 год.

Игроки и критики чаще оставались довольны и уходили с хорошими впечатлениями. Так всего менее 1% у пользователей и критиков оставили низкую оценку, когда как больше 30% имеют оценку "средняя". Это не плохой показатель, учитывая большое количество платформ и жанров используемых для анализа этого периода.

"Нулевые" стали для Play Station 2 и Nintendo DS самыми "игродельными". Больше всего продуктов вышло именно на этих платформах.

Критики выше всего оценили платформы: Play Station 2 и 3 поколения, PC (Персональный компьютер) и Xbox 360. А пользователи отдали предпочтение Play Station 2, PC и Xbox. 

Количество высоких оценок к 2013 году постепенно уменьшается, это может быть связано с тем, что запросы к геймдизайну/сюжету/механикам у конечного пользователя из года в год повышается. Если построить кривую популярности топовых  жанров Экшен игр, Спортивных симуляторов, Шутеров и Ролевых игр, то получится "волна". Видимо удачные и менее интересные продукты чередуют друг друга каждый год.